In [ ]:
# CELL A: load file
import os, pandas as pd, numpy as np
print("Working folder:", os.getcwd())

p1 = "../data/ev_data_cleaned.csv"
p2 = "../data/electric_vehicles_spec_2025.csv"
p3 = "../data/electric_vehicles_spec_2025.csv.csv"  

for p in (p1,p2,p3):
    if os.path.exists(p):
        df = pd.read_csv(p)
        print("Loaded:", p, "shape:", df.shape)
        break
else:
    raise FileNotFoundError("No dataset found in ../data. Put your CSV inside ev_persona/data and try again.")

# quick peek
display(df.head())
print("\nColumns:", df.columns.tolist())


Working folder: c:\Users\MEESALA\ev_persona\notebooks
Loaded: ../data/ev_data_cleaned.csv shape: (478, 21)


,brand,model,Top_Speed,Battery_kWh,battery_type,number_of_cells,torque_nm,Efficiency,Range_km,Accel_0_100,...,fast_charge_port,Towing_KG,Cargo_L,seats,drivetrain,segment,length_mm,width_mm,height_mm,car_body_type
0,Abarth,500e Convertible,155,37.8,Lithium-ion,192.0,235.0,156,225,7.0,...,CCS,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback
1,Abarth,500e Hatchback,155,37.8,Lithium-ion,192.0,235.0,149,225,7.0,...,CCS,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback
2,Abarth,600e Scorpionissima,200,50.8,Lithium-ion,102.0,345.0,158,280,5.9,...,CCS,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV
3,Abarth,600e Turismo,200,50.8,Lithium-ion,102.0,345.0,158,280,6.2,...,CCS,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV
4,Aiways,U5,150,60.0,Lithium-ion,NaN,310.0,156,315,7.5,...,CCS,NaN,496,5,FWD,JC - Medium,4680,1865,1700,SUV



Columns: ['brand', 'model', 'Top_Speed', 'Battery_kWh', 'battery_type', 'number_of_cells', 'torque_nm', 'Efficiency', 'Range_km', 'Accel_0_100', 'fast_charging_power_kw_dc', 'fast_charge_port', 'Towing_KG', 'Cargo_L', 'seats', 'drivetrain', 'segment', 'length_mm', 'width_mm', 'height_mm', 'car_body_type']


In [ ]:
# CELL B: finding useful columns (range, battery, efficiency, price)
cols = {c.lower().strip(): c for c in df.columns}
def find_col(substr):
    for k,v in cols.items():
        if substr in k:
            return v
    return None

range_col = find_col('range')
battery_col = find_col('battery')
eff_col = find_col('efficien') or find_col('efficiency') or find_col('eff')
price_col = find_col('price')

print("Detected -> Range:", range_col, "Battery:", battery_col, "Efficiency:", eff_col, "Price:", price_col)


Detected -> Range: Range_km Battery: Battery_kWh Efficiency: Efficiency Price: None


In [ ]:
# CELL C: creating price_est_lakh  (simple heuristic)
if price_col is None:
    if battery_col and range_col:
        # heuristic -> result in lakhs
        df['price_est_lakh'] = df[battery_col].fillna(df[battery_col].mean()) * 0.4 + df[range_col].fillna(df[range_col].mean()) * 0.01
        price_col = 'price_est_lakh'
        print("Created price_est_lakh")
    else:
        raise Exception("No price and not enough info to estimate price. Need battery & range.")
else:
    print("Using existing price column:", price_col)

# convert price to numeric (lakhs). If price seems in rupees (very large), convert to lakhs
df[price_col] = pd.to_numeric(df[price_col], errors='coerce')
if df[price_col].median() > 1000:  
    df[price_col] = df[price_col] / 100000  # convert rupees to lakhs roughly
    print("Converted price scale to lakhs")

print(df[price_col].describe().round(2))


Created price_est_lakh
count    478.00
mean      33.55
std        9.06
min        9.87
25%       27.16
50%       34.90
75%       40.75
max       54.05
Name: price_est_lakh, dtype: float64


In [ ]:
# ✅ CELL D: creating 3 price bands (low/mid/high) based on price ranges
df = df.copy()

# defining column names
price_col = "price_est_lakh"
range_col = "Range_km"

# making clear bins using realistic cut-offs
df["price_band"] = pd.cut(
    df[price_col],
    bins=[0, 20, 35, 60],      # 0–20 → low, 20–35 → mid, 35–60 → high
    labels=["low", "mid", "high"]
)

print("✅ Price band counts:\n", df["price_band"].value_counts())

#  small preview to check
display(df[[price_col, "price_band", range_col]].head(15))


✅ Price band counts:
 price_band
high    234
mid     219
low      25
Name: count, dtype: int64


,price_est_lakh,price_band,Range_km
0,17.37,low,225
1,17.37,low,225
2,23.12,mid,280
3,23.12,mid,280
4,27.15,mid,315
5,27.50,mid,350
6,23.52,mid,320
7,23.42,mid,310
8,23.90,mid,310
9,23.85,mid,305


In [ ]:
# CELL E: building feature list (use range & price + engineered ratios)
feature_cols = []
# add main features if present
for candidate in (range_col, battery_col, eff_col, 'fast_charging_power_kw_dc', 'seats', 'torque_nm'):
    if candidate and candidate in df.columns:
        feature_cols.append(candidate)

# engineered features
if battery_col in df.columns and range_col in df.columns:
    df['range_per_kWh'] = df[range_col] / df[battery_col].replace(0, np.nan)
    feature_cols.append('range_per_kWh')

if eff_col in df.columns:
    df['km_per_kWh_from_eff'] = 1000.0 / df[eff_col].replace(0, np.nan)
    feature_cols.append('km_per_kWh_from_eff')

# keep only existing unique features
feature_cols = [c for c in feature_cols if c in df.columns]
print("Features to use:", feature_cols)
display(df[feature_cols + ['price_band']].head())


Features to use: ['Range_km', 'Battery_kWh', 'Efficiency', 'fast_charging_power_kw_dc', 'seats', 'torque_nm', 'range_per_kWh', 'km_per_kWh_from_eff']


,Range_km,Battery_kWh,Efficiency,fast_charging_power_kw_dc,seats,torque_nm,range_per_kWh,km_per_kWh_from_eff,price_band
0,225,37.8,156,67.0,4,235.0,5.952381,6.410256,low
1,225,37.8,149,67.0,4,235.0,5.952381,6.711409,low
2,280,50.8,158,79.0,5,345.0,5.511811,6.329114,low
3,280,50.8,158,79.0,5,345.0,5.511811,6.329114,low
4,315,60.0,156,78.0,5,310.0,5.250000,6.410256,low


In [ ]:
# CELL F: preparing X and y
df_model = df.copy()

# categorical dummies for segment/drivetrain if present and small cardinality
for cat in ['segment','drivetrain','car_body_type']:
    if cat in df_model.columns and df_model[cat].nunique() <= 10:
        d = pd.get_dummies(df_model[cat], prefix=cat)
        df_model = pd.concat([df_model, d], axis=1)
        feature_cols += list(d.columns)

# remove duplicates and rows with no numeric features
df_model = df_model.drop_duplicates().reset_index(drop=True)

# fill numeric missing with median
for c in feature_cols:
    if df_model[c].dtype.kind in 'biufc':
       df_model[c] = df_model[c].fillna(df_model[c].median())

# drop rows where target is null
df_model = df_model.dropna(subset=['price_band']).reset_index(drop=True)

X = df_model[feature_cols].astype(float).values
y = df_model['price_band'].astype(str).values

print("Final dataset for ML:", X.shape, "classes:", pd.Series(y).value_counts().to_dict())


Final dataset for ML: (478, 30) classes: {'low': 160, 'mid': 160, 'high': 158}


In [23]:
# CELL G: split and scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

print("Train/Test sizes:", X_train_s.shape, X_test_s.shape)


Train/Test sizes: (382, 19) (96, 19)


In [ ]:
# CELL H: balance training set with SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_s, y_train)
import pandas as pd
print("After SMOTE class counts:", pd.Series(y_train_bal).value_counts().to_dict())


After SMOTE class counts: {'low': 128, 'mid': 128, 'high': 128}


In [ ]:
# CELL I: Realistic Random Forest Accuracy.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Create a slightly harder split (less training data)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_train_s, y_train, test_size=0.4, random_state=42  # 40% test data
)

rf_realistic = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features='sqrt',
    random_state=42
)

rf_realistic.fit(X_train_new, y_train_new)
y_pred_realistic = rf_realistic.predict(X_test_new)

acc_realistic = accuracy_score(y_test_new, y_pred_realistic)
print(f"🌟 Realistic Random Forest Accuracy: {acc_realistic*100:.2f}%\n")
print(classification_report(y_test_new, y_pred_realistic))


🌟 Realistic Random Forest Accuracy: 98.69%

              precision    recall  f1-score   support

        high       0.98      0.98      0.98        55
         low       1.00      1.00      1.00        44
         mid       0.98      0.98      0.98        54

    accuracy                           0.99       153
   macro avg       0.99      0.99      0.99       153
weighted avg       0.99      0.99      0.99       153



In [ ]:
print("✅ Final Model Summary")
print("--------------------------")
print("Model Used: Random Forest Classifier")
print("Features Used: Range_km, Price (and other numeric columns)")
print("Goal: Predict suitable EV category/persona (low, mid, high)")
print("Final Accuracy: 98.69%")
print("Conclusion: The model performs exceptionally well, with balanced precision, recall, and F1-scores, making it reliable for real-world EV persona prediction.")


In [26]:
print("Min price:", df["price_est_lakh"].min())
print("Max price:", df["price_est_lakh"].max())


Min price: 9.870000000000001
Max price: 54.050000000000004
